In [1]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import cv2
from IPython.display import display
import math, random
import time, datetime, sys, os, shutil
import operator

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import pandas as pd

#tf.config.optimizer.set_jit(True)
#tf.debugging.set_log_device_placement(True)
print(tf.config.experimental.list_physical_devices())

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

%run ./variables.ipynb
%run ./utils.ipynb
%run ../utils/data_utils.ipynb

id_map = get_selected_taxons(SELECTED_TAXONS)
id_map_inv = get_selected_taxons(SELECTED_TAXONS, inv=True)
n_classes = len(list(id_map.keys()))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Handle data

## Preparing panda arrays

In [2]:
X, y, _ = get_dataset(ids=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Original length:", len(X))
print("Test length:", len(X_train), "- n classes:", len(np.unique(y_train)))
print("Test length:", len(X_test), "- n classes:", len(np.unique(y_test)))
# Balance dataset
train_dict = {}
for file, label in zip(X_train, y_train):
    train_dict.setdefault(label, []).append(file)
max_samples = np.max([len(train_dict[taxon_id]) for taxon_id in train_dict])
X_train = []
y_train = []
for taxon_id in train_dict:
    ratio = np.ceil(max_samples/len(train_dict[taxon_id]))
    tmp = np.repeat(train_dict[taxon_id], ratio)
    np.random.shuffle(tmp)
    train_dict[taxon_id] = tmp[0:max_samples]
    X_train.extend(tmp[0:max_samples])
    y_train.extend([taxon_id]*max_samples)

'171/171'

Original length: 6144
Test length: 4915 - n classes: 171
Test length: 1229 - n classes: 167


In [3]:
data_train = {'png_path':  X_train, 'taxon_id': y_train}
data_test = {'png_path':  X_test, 'taxon_id': y_test}

df_train = pd.DataFrame(data_train, columns = ['png_path', 'taxon_id'])
df_test = pd.DataFrame(data_test, columns = ['png_path', 'taxon_id'])
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

# Prtining some infos
print(len(df_train), len(df_test))
df_train.head()
df_test.head()

20691 1229


,png_path,taxon_id
0,/mnt/nvme-storage/pfauregi/datasets/condensed/...,NERI
1,/mnt/nvme-storage/pfauregi/datasets/condensed/...,GMIN
2,/mnt/nvme-storage/pfauregi/datasets/condensed/...,NANT
3,/mnt/nvme-storage/pfauregi/datasets/condensed/...,CMED
4,/mnt/nvme-storage/pfauregi/datasets/condensed/...,CMLF


## Setting up the flows

In [4]:
datagen = ImageDataGenerator(rescale=1./255.,
                         rotation_range=90, 
                         brightness_range=[0.8,1.2], 
                         horizontal_flip=True, 
                         vertical_flip=True,
                         fill_mode='nearest',
                         width_shift_range=40,
                         height_shift_range=40,
                         zoom_range=0.2,
                         validation_split=0.2,
                         #featurewise_std_normalization=True,
                         data_format="channels_last")

classes_array = get_id_array(id_map)
train_generator = datagen.flow_from_dataframe(
        dataframe=df_train,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        classes=classes_array,
        class_mode='categorical')
val_generator = datagen.flow_from_dataframe(
        dataframe=df_test,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        classes=classes_array,
        class_mode='categorical')

train_spe = train_generator.samples // BATCH_SIZE
val_spe = val_generator.samples // BATCH_SIZE

Found 20691 validated image filenames belonging to 230 classes.
Found 1229 validated image filenames belonging to 230 classes.


## Testing

In [ ]:
i = 0
stop = False
for batch in train_generator:
    images = batch[0]
    labels = batch[1]
    for i in range(images.shape[0]):
        print(np.argmax(labels[i]))
        image = (images[i,:,:,:]*255).astype('uint8')
        #print(image)
        display(Image.fromarray(image))
        i+=1
        if i>=1:
            stop = True
            break
    if stop: break;
    #display()

# Model desgin

In [5]:
# fetching base model
#base_model = Xception(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling=None)
input_tensor = Input(shape=(256, 256, 3))
base_model = InceptionV3(weights='imagenet', input_tensor=input_tensor, include_top=False)

In [6]:
# setting model for specifiv case
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(230, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

# Training

In [7]:
delete_all_files_in_folder(LOG_DIR)
delete_all_files_in_folder(SAVED_MODELS_ROOT)

## Setting callbacks

In [8]:
# Setting tensorboard
log_dir = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
log_file=os.path.join(SAVED_MODELS_ROOT, "model.log")
csv_logger = CSVLogger(log_file)

## New layers only

In [10]:
n_epochs = 3
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

3 epochs composed of 646 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 646 steps, validate for 38 steps
Epoch 1/3
646/646 [==============================] - 276s 427ms/step - loss: 2.7110 - accuracy: 0.3761 - val_loss: 4.6534 - val_accuracy: 0.0995
Epoch 2/3
646/646 [==============================] - 267s 414ms/step - loss: 1.3030 - accuracy: 0.6328 - val_loss: 4.8508 - val_accuracy: 0.1431
Epoch 3/3
646/646 [==============================] - 268s 415ms/step - loss: 1.0339 - accuracy: 0.6904 - val_loss: 5.0072 - val_accuracy: 0.1530


## Training 2 last inceptions blocks

In [12]:
n_epochs = 7
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False,
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

7 epochs composed of 646 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 646 steps, validate for 38 steps
Epoch 3/9
646/646 [==============================] - 272s 421ms/step - loss: 0.9006 - accuracy: 0.7223 - val_loss: 4.2861 - val_accuracy: 0.2706
Epoch 4/9
646/646 [==============================] - 268s 414ms/step - loss: 0.4783 - accuracy: 0.8431 - val_loss: 5.9722 - val_accuracy: 0.2311
Epoch 5/9
646/646 [==============================] - 267s 414ms/step - loss: 0.3722 - accuracy: 0.8778 - val_loss: 4.0993 - val_accuracy: 0.2878
Epoch 6/9
646/646 [==============================] - 268s 415ms/step - loss: 0.3143 - accuracy: 0.8935 - val_loss: 3.3611 - val_accuracy: 0.3322
Epoch 7/9
646/646 [==============================] - 268s 415ms/step - loss: 0.2674 - accuracy: 0.9116 - val_loss: 4.6790 - val_accuracy: 0.2895
Epoch 8/9
646/646 [==============================] - 268s 415ms/step - loss: 0.2477 - accuracy: 0.9194 - val_loss: 4.2780 - val

## Training full model

In [13]:
n_epochs = 20
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

20 epochs composed of 646 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 646 steps, validate for 38 steps
Epoch 9/28
646/646 [==============================] - 273s 422ms/step - loss: 0.8852 - accuracy: 0.7378 - val_loss: 1.4018 - val_accuracy: 0.6472
Epoch 10/28
646/646 [==============================] - 265s 411ms/step - loss: 0.4395 - accuracy: 0.8520 - val_loss: 1.0385 - val_accuracy: 0.7574
Epoch 11/28
646/646 [==============================] - 265s 410ms/step - loss: 0.3543 - accuracy: 0.8825 - val_loss: 0.9637 - val_accuracy: 0.7229
Epoch 12/28
646/646 [==============================] - 267s 413ms/step - loss: 0.2891 - accuracy: 0.9029 - val_loss: 1.3534 - val_accuracy: 0.6826
Epoch 13/28
646/646 [==============================] - 266s 412ms/step - loss: 0.2692 - accuracy: 0.9112 - val_loss: 1.0368 - val_accuracy: 0.7590
Epoch 14/28
646/646 [==============================] - 266s 412ms/step - loss: 0.2342 - accuracy: 0.9234 - val_loss: 

## Saving model

In [14]:
save_model(model, SAVED_MODELS_ROOT)

Saved model to ./saved_models/model.json
Saved weights to ./saved_models/model.h5
